Cryptocurrency trading bot.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_base.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger base class.

# Library imports.
from cryptocurrency.resample import resample
from binance.client import Client
from abc import abstractmethod, ABC
from time import sleep, time
from os.path import exists, join
from os import mkdir

import pandas as pd

class Crypto_logger_base(ABC):
    def __init__(self, interval='15s', delay=4.7, buffer_size=3000, directory='crypto_logs', 
                 log_name='crypto_log', raw=False):
        """
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param delay: delay between Binance API requests. Minimum calculated was 4.7 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param directory: the directory where to output the logs.
        :param log_name: name of the log file.
        :param raw: whether the log dumps raw (instantaneous) or OHLCV data.
        """
        self.interval = interval
        self.delay = delay
        self.buffer_size = buffer_size
        self.directory = directory
        self.raw = raw

        self.log_name = join(self.directory, log_name + '.txt')
        self.log_screened_name = join(self.directory, log_name + '_screened.txt')

        if not exists(self.directory):
            mkdir(self.directory)

    #self.get_from_file(log_name=self.log_name, from_raw=False)
    #self.get_from_file(log_name=self.input_log_name, from_raw=self.load_from_ohlcv)
    def get_from_file(self, log_name, from_raw=False):
        if from_raw:
            dataset = pd.read_csv(log_name, header=0, index_col=0)
        else:
            dataset = pd.read_csv(log_name, header=[0, 1], index_col=0)
        dataset.index = pd.DatetimeIndex(dataset.index)
        return dataset.sort_index(axis='index')

    @abstractmethod
    def get(self, **kwargs):
        raise NotImplementedError()

    @abstractmethod
    def screen(self, **kwargs):
        raise NotImplementedError()

    def put(self, dataset):
        dataset = dataset.copy().reset_index()
        if self.raw:
            dataset = dataset.drop_duplicates(subset=['symbol', 'count'], 
                                              keep='first', ignore_index=True)
        else:
            dataset = dataset.drop_duplicates(keep='last', ignore_index=True)

        if 'date' in dataset.columns:
            min_index_int = dataset[dataset['date'] == self.min_index].index[0]
            dataset = dataset.set_index('date')
        if not self.raw:
            dataset = resample(dataset, self.interval)
        if 'date' in dataset.columns:
            dataset = dataset.iloc[min_index_int:]

        dataset = dataset.tail(self.buffer_size)
        dataset.to_csv(self.log_name)
        self.min_index = dataset.index[0]
        return dataset

    def start(self, append=False, roll=0):
        """Main logger loop."""
        print('Starting crypto logger.')

        if exists(self.log_name) and 'output' in self.log_name:
            self.dataset = self.get_from_file(log_name=self.log_name, from_raw=False)
            self.dataset = self.dataset.tail(self.buffer_size)
        else:
            self.dataset = self.get()

        self.min_index = self.dataset.index[-1]
        self.dataset = self.put(self.dataset)

        while True:
            try:
                dataset = pd.concat([self.dataset, self.get()], axis='index', join='outer')
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                self.dataset = self.put(dataset)
            except (KeyboardInterrupt, SystemExit):
                print('Saving latest complete dataset...')
                self.dataset = self.put(dataset)
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                dataset_screened = self.screen(self.dataset)
                if dataset_screened is not None:
                    if roll != 0:
                        if append and exists(self.log_screened_name):
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    elif append:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    else:
                        dataset_screened.to_csv(self.log_screened_name)
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            sleep(self.delay)
        print('Crypto logger process done.')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_output.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for arbitrary intervals.

# Library imports.
#from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.indicators import filter_in_market, screen_one
from os.path import exists, join

import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_output(Crypto_logger_base):
    def __init__(self, delay=12, interval_input='15s', interval='15s', buffer_size=60, 
                 input_log_name='input'):
        """
        :param delay: delay between logger/screener service interruptions.
        :param interval_input: OHLCV interval from input log. Default is 15 seconds.
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param buffer_size: buffer size to avoid crashing on memory accesses.
        :param input_log_name: the directory where to take the logs from.
        """
        self.data_before = pd.DataFrame()
        input_log_name = 'crypto_' + input_log_name + '_log_'
        self.load_from_ohlcv = interval_input != interval
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_output_log_' + interval, 
                         raw=False)

        self.input_log_name = \
            join(self.directory, input_log_name + interval_input + '.txt')
        self.input_log_screened_name = \
            join(self.directory, input_log_name + interval_input + '_screened.txt')

    def screen(self, dataset):
        input_filtered = None
        if exists(self.input_log_screened_name):
            input_filtered = pd.read_csv(self.input_log_screened_name, header=0, index_col=0)
            input_filter = set(input_filtered['symbol'].tolist())
            old_columns = set(dataset.columns.get_level_values(0).tolist())
            new_columns = list(input_filter & old_columns)
            dataset = dataset[new_columns]
            assets = filter_in_market(screen_one, dataset)
            input_filtered = input_filtered[input_filtered['symbol'].isin(assets)]
        return input_filtered

    def resample_from_raw(self, df):
        df = df[['symbol', 'close', 'rolling_base_volume', 'rolling_quote_volume']]
        df['base_volume'] = df['rolling_base_volume'].copy()
        df['quote_volume'] = df['rolling_quote_volume'].copy()
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['close', 'rolling_base_volume', 
                                    'rolling_quote_volume', 
                                    'base_volume', 'quote_volume'], 
                            aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                     'base_volume': 'max', 'quote_volume': 'max', 
                                     'rolling_base_volume': 'max', 
                                     'rolling_quote_volume': 'max'})
        df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) 
                                                for col in df.columns.values], 
                                               names=('pair', 'symbol'))
        df = df.rename(columns={'close_first': 'open', 'close_max': 'high', 
                                'close_min': 'low', 'close_last': 'close', 
                                'base_volume_max': 'base_volume', 
                                'quote_volume_max': 'quote_volume', 
                                'rolling_base_volume_max': 'rolling_base_volume', 
                                'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                       level=0)
        df['base_volume'] = df['base_volume'].fillna(0)
        df['quote_volume'] = df['quote_volume'].fillna(0)
        df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad').fillna(0)
        df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad').fillna(0)
        df = df.sort_index().iloc[1:]
        df.columns = df.columns.swaplevel(0, 1)
        return df

    def get(self):
        dataset = self.get_from_file(log_name=self.input_log_name, 
                                     from_raw=not self.load_from_ohlcv)
        if not self.load_from_ohlcv:
            dataset = self.resample_from_raw(dataset)
        return dataset.tail(2)

Main.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_output_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_output import Crypto_logger_output

crypto_logger_output_1min = Crypto_logger_output(delay=33, 
                                                 interval_input='15s', 
                                                 interval='15s', 
                                                 buffer_size=60, 
                                                 input_log_name='output')
crypto_logger_output_1min.start(append=False, roll=1000)

In [ ]:
import pandas as pd
input_log_name = 'crypto_logs/crypto_input_log_15s.txt'
df = pd.read_csv(input_log_name, header=0, index_col=0)
df.index = pd.DatetimeIndex(df.index)
df = df.sort_index(axis='index')
#df = crypto_logger_output_1min.resample_from_raw(df)
df

In [ ]:
import pandas as pd
#crypto_output_log_1d = crypto_logger_output_1d.log_name
crypto_output_log_1d = 'crypto_logs/crypto_output_log_1d.txt'
df_1d = pd.read_csv(crypto_output_log_1d, header=[0, 1], index_col=0)
df_1d.index = pd.DatetimeIndex(df_1d.index)
df_1d = df_1d.sort_index(axis='index')
df_1d['BTC'].tail(20)

In [1]:
import pandas as pd
#crypto_output_log_1h = crypto_logger_output_1h.log_name
crypto_output_log_1h = 'crypto_logs/crypto_output_log_1h.txt'
df_1h = pd.read_csv(crypto_output_log_1h, header=[0, 1], index_col=0)
df_1h.index = pd.DatetimeIndex(df_1h.index)
df_1h = df_1h.sort_index(axis='index')
df_1h['BTC'].tail(20)

pair,open,high,low,close,base_volume,quote_volume,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-11-12 09:00:00,16841.35,16905.22,16800.00,16863.98,1.961440e+08,1.960534e+08,8.580557e+09,8.573127e+09
2022-11-12 10:00:00,16864.46,16977.45,16807.13,16843.88,2.522414e+08,2.527530e+08,8.533088e+09,8.526359e+09
2022-11-12 11:00:00,16842.66,16887.51,16790.30,16866.92,1.583291e+08,1.581320e+08,8.380402e+09,8.373272e+09
2022-11-12 12:00:00,16866.46,16918.29,16809.83,16862.99,1.680811e+08,1.681323e+08,8.265257e+09,8.259148e+09
2022-11-12 13:00:00,16863.46,16869.00,16757.00,16840.79,1.690897e+08,1.689236e+08,7.994280e+09,7.985305e+09
2022-11-12 14:00:00,16839.75,16964.90,16799.28,16900.01,2.279972e+08,2.277357e+08,6.714130e+09,6.712341e+09
2022-11-12 15:00:00,16901.33,16996.35,16873.84,16935.74,2.037338e+08,2.038166e+08,6.220490e+09,6.217950e+09
2022-11-12 16:00:00,16935.74,16964.37,16870.01,16888.68,1.503602e+08,1.505260e+08,5.875942e+09,5.875519e+09
2022-11-12 17:00:00,16889.57,16910.83,16857.47,16891.69,1.320677e+08,1.320214e+08,5.646655e+09,5.647117e+09


In [ ]:
import pandas as pd
#crypto_output_log_1min = crypto_logger_output_1min.log_name
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min = df_1min.sort_index(axis='index')
df_1min['BTC'].tail(20)

In [ ]:
dataset = crypto_logger_output_1min.dataset
dataset

In [ ]:
dataset['BTCUSDT']

In [ ]:
dataset['ZILBUSD']

In [7]:
import pandas_ta as ta

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_relative_volume_levels_smoothed_trigger(data, average1=26, average2=14):
    volume = data['volume']
    volume_average = ta.sma(close=volume, length=average1)
    relative_volume = volume / volume_average
    smoothed_relative_volume = ta.sma(close=relative_volume, length=average2)
    return (smoothed_relative_volume > threshold).iat[-1]

def get_relative_volume_levels_at_time_smoothed_thresholded(data):
    try:
        volume = data['volume']
        #volume = volume.groupby(pd.Grouper(freq='D')).cumsum()
        cum_volume = volume.groupby(pd.Grouper(freq='24h')).cumsum()
        #volume = volume.groupby(pd.Grouper(freq='60m')).cumsum()
        cum_rvol = (cum_volume / cum_volume.shift(1)).fillna(method='pad')
        rvol = (volume / volume.shift(1)).fillna(method='pad')
        bar_up = (data['close'] > data['open'])
        bar_up |= (data['close'] == data['open']) & (data['close'].diff() > 0)
        bar_up = bar_up.astype(int)
        bar_up = bar_up * 2 - 1
        #rvol *= bar_up
        cum_rvol_dir = cum_rvol * bar_up
        rvol_dir = rvol * bar_up
        rvol_indicator = ta.hma(rvol, length=14, talib=True)
        rvol_dir_indicator = ta.hma(rvol_dir, length=14, talib=True)
        cum_rvol_indicator = ta.hma(cum_rvol, length=14, talib=True)
        cum_rvol_dir_indicator = ta.hma(cum_rvol_dir, length=14, talib=True)
        rvol_indicator = rvol_indicator.rename('relative_volume_levels_smoothed')
        rvol_dir_indicator = rvol_dir_indicator.rename('relative_volume_levels_dir_smoothed')
        cum_rvol_indicator = cum_rvol_indicator.rename('cum_relative_volume_levels_smoothed')
        cum_rvol_dir_indicator = cum_rvol_dir_indicator.rename('cum_relative_volume_levels_dir_smoothed')
        #threshold = (ta.sma(rvol, length=100, talib=True) + ta.stdev(rvol, length=100, talib=True))
        threshold_dir = 0
        threshold = 2
        rvol_thresholded = (rvol_indicator > threshold).iat[-1]
        rvol_dir_thresholded = (rvol_dir_indicator > threshold_dir).iat[-1]
        cum_rvol_thresholded = (cum_rvol_indicator > threshold).iat[-1]
        cum_rvol_dir_thresholded = (cum_rvol_dir_indicator > threshold_dir).iat[-1]
        trigger = (rvol_thresholded | rvol_dir_thresholded | cum_rvol_thresholded | cum_rvol_dir_thresholded)
    except Exception as e:
        print('rvol exception:', e)
        trigger = False
    return trigger

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iat[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iat[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iat[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iat[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iat[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iat[-2]) or \
            (MACD['MACD_12_26_9'].iat[-1] > MACD['MACDs_12_26_9'].iat[-1]))

def get_not_square_wave_triggers(data, multiplier_schedule):
    triggers = True
    for multiplier in multiplier_schedule:
        period_1 = -4 * multiplier
        uniques_1 = 2 * multiplier
        square_wave_trigger_1 = (data.iloc[period_1:]['close'].unique().size < uniques_1)
        if square_wave_trigger_1:
            triggers = False
            break
        else:
            period_2 = -15 * multiplier
            uniques_2 = 6 * multiplier
            square_wave_trigger_2 = (data.iloc[period_2:]['close'].unique().size < uniques_2)
            if square_wave_trigger_2:
                triggers = False
                break
    return triggers

def get_minute_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1, 2, 3, 5, 10, 15, 20, 45])

def get_hourly_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_not_square_wave_triggers(data):
    return get_not_square_wave_triggers(data, multiplier_schedule=[1])

def get_daily_volume_minimum_trigger(data):
    return (data['volume'] > 1000000).iat[-1]

def get_daily_volume_change_trigger(data):
    return ((data['volume'].pct_change(1) * 100) > 300).iat[-1]

def get_minute_daily_volume_minimum_trigger(data):
    return (data['rolling_base_volume'] > 1000000).iat[-1]

def get_minute_daily_volume_change_trigger(data):
    return ((data['rolling_base_volume'].pct_change(1440) * 100) > 300).iat[-1]

def get_rising_volume_trigger(data):
    return (data['rolling_base_volume'].diff(1) > 0).iat[-1]

In [ ]:
df = df_1d.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_daily_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_relative_volume_levels_smoothed_trigger, df).tolist())
filtered_1d = pd.Series(list(filtered_1 & filtered_2 & filtered_3))
filtered_1d

In [9]:
df = df_1h.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_hourly_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_relative_volume_levels_at_time_smoothed_thresholded, df).tolist())
filtered_1h = pd.Series(list(filtered_1 & filtered_2))
filtered_1h

0        XRP
1        PSG
2        MOB
3       AUTO
4        HNT
       ...  
296    BTCST
297    TFUEL
298      MDT
299      BAR
300      XMR
Length: 301, dtype: object

In [ ]:
from cryptocurrency.indicators import filter_in_market, screen_one
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered = filter_in_market(screen_one, df).tolist()
filtered

In [ ]:
df = df_1min.copy()
df.columns = df.columns.swaplevel(0, 1)
df = df.rename(columns={'base_volume': 'volume'})
df.columns = df.columns.swaplevel(0, 1)
filtered_1 = set(filter_in_market(get_minute_not_square_wave_triggers, df).tolist())
filtered_2 = set(filter_in_market(get_minute_daily_volume_minimum_trigger, df).tolist())
filtered_3 = set(filter_in_market(get_minute_daily_volume_change_trigger, df).tolist())
filtered_4 = set(filter_in_market(get_rising_volume_trigger, df).tolist())
filtered_1min = pd.Series(list(filtered_1 & filtered_2 & filtered_3 & filtered_4))
filtered_1min

In [ ]:
list(set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1d.tolist()))

In [ ]:
list(set(filtered_1min.tolist()) & set(filtered_1h.tolist()) & set(filtered_1d.tolist()))

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
import pandas as pd

crypto_exchange_info = 'crypto_logs/crypto_exchange_info.txt'
exchange_info = pd.read_csv(crypto_exchange_info, header=0, index_col=0)
exchange_info

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info)
#conversion_table.sort_values(by='rolling_traded_volume', ascending=False).reset_index(drop=True).head(50)

In [ ]:
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table

offset_s = get_timezone_offset_in_seconds()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, as_pair=False)
conversion_table.sort_values(by='rolling_base_volume', ascending=False).reset_index(drop=True).head(50)

In [ ]:
conversion_table['quote_asset'].unique()

In [ ]:
def get_connected_assets(asset, exchange_info, priority='accuracy'):
    def reorder(connected_assets, priority):
        prioritized = [asset for asset in priority if asset in connected_assets]
        order = {asset: i for i, asset in enumerate(prioritized)}
        connected_assets_items = [asset for asset in connected_assets if asset in order]
        connected_assets_items.sort(key=order.get)
        connected_assets_iter = iter(connected_assets_items)
        return [next(connected_assets_iter) if asset in order 
                else asset for asset in connected_assets]
    priorities = {}
    priorities['accuracy'] = ['USDT', 'BTC', 'BUSD', 'ETH', 'BNB', 'AUD']
    priorities['fees'] = ['BUSD', 'BTC', 'BNB', 'ETH', 'USDT', 'AUD']
    priorities['wallet'] = ['BTC', 'ETH', 'BUSD', 'BNB', 'USDT', 'AUD']
    priority = priorities[priority]
    connected_base_assets = exchange_info['base_asset'] == asset
    connected_base_assets = exchange_info[connected_base_assets]
    connected_base_assets = connected_base_assets['quote_asset'].tolist()
    connected_quote_assets = exchange_info['quote_asset'] == asset
    connected_quote_assets = exchange_info[connected_quote_assets]
    connected_quote_assets = connected_quote_assets['base_asset'].tolist()
    connected_assets = sorted(list(set(connected_base_assets + connected_quote_assets)))
    prioritized = reorder(connected_assets, priority)
    return prioritized

asset = 'AXS'
connected_assets = get_connected_assets(asset, exchange_info, priority='fees')
print(connected_assets)

In [ ]:
symbol = 'NEBL'

In [ ]:
import pandas as pd
crypto_output_log_1d = 'crypto_logs/crypto_output_log_1d.txt'
df_1d = pd.read_csv(crypto_output_log_1d, header=[0, 1], index_col=0)
df_1d.index = pd.DatetimeIndex(df_1d.index)
df_1d

In [ ]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

In [ ]:
df_15s[symbol]

In [ ]:
import pandas as pd
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min

In [ ]:
df_1min[symbol].tail(20)

In [ ]:
import pandas as pd
crypto_output_log_1h = 'crypto_logs/crypto_output_log_1h.txt'
df_1h = pd.read_csv(crypto_output_log_1h, header=[0, 1], index_col=0)
df_1h.index = pd.DatetimeIndex(df_1h.index)
df_1h

In [ ]:
df_1h[symbol].tail(20)

In [ ]:
df = df_1h.copy()
df

In [ ]:
df.tz_local

In [ ]:
import time
df = df_1h.copy()
is_dst = time.localtime().tm_isdst
timezone = time.tzname[is_dst]
offset_ns = time.altzone if is_dst else time.timezone
offset = (offset_ns / 60 / 60 * -1)
df.index = df.index + pd.Timedelta(-is_dst, unit='h')
df.index = df.index.tz_localize(tz=time.tzname[0])
df.index = pd.DatetimeIndex(df.index, ambiguous='infer')
df.index = df.index.tz_convert('UTC')
df.index

In [ ]:
df.index[-1].dst()

In [ ]:
import datetime
df.index.to_pydatetime()[-1]

In [ ]:
df = df_1min.copy()
df

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion_table import get_conversion_table
import pandas as pd

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)

conversion_table = df_1min.copy()
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table = conversion_table.drop(columns=['rolling_base_volume', 
                                                  'rolling_quote_volume'])
conversion_table.columns = conversion_table.columns.swaplevel(0, 1)
conversion_table

In [ ]:
from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets

new_conversion_table = convert_ohlcvs_from_pairs_to_assets(conversion_table, 
                                                           exchange_info)
new_conversion_table

In [ ]:
new_conversion_table['ETH']